In [ ]:
# from typing import List, Any
# from pydantic import BaseModel

# def get_name(metric_str):
#     if 'Range Invariant Multiscale SSIM' in metric_str:
#         return 'RISSIM'
#     elif 'Multiscale SSIM' in metric_str:
#         return 'MSSIM'
#     elif 'PSNR' in metric_str:
#         return 'PSNR'
#     else:
#         raise ValueError("Unknown metric name in str:\t", metric_str)

# def get_metric_tokens(metric_str):
#     tokens = [x for x in metric_str.split(" ") if x != '']
#     tokens = [x for x in tokens if '+-' in x]
#     return tokens

# class Metric(BaseModel):
#     name: str  = ""
#     mean: List[float]  = []
#     std: List[float]  = []

#     @classmethod
#     def from_str(cls, value:str) -> Any:
#         if isinstance(value, str):
#             name = get_name(value)
#             tokens = get_metric_tokens(value)
#             mean_data = [float(x.split('+')[0]) for x in tokens]
#             std_data = [float(x.split('-')[1]) for x in tokens]
#             return cls(name=name, mean=mean_data, std=std_data)

#         return value

# Metric.from_str('Range Invariant Multiscale SSIM on Highres 0.988+-0.004 0.995+-0.001    0.994+-0.0044')

In [ ]:
import pickle
from collections import defaultdict

keys = {"2ms":"2406/D25-M3-S0-L8/4",
        "3ms":"2406/D25-M3-S0-L8/5",
        "5ms":"2406/D25-M3-S0-L8/6",
        "20ms":"2406/D25-M3-S0-L8/14",
        "500ms":"2406/D25-M3-S0-L8/17"}

output_data_dir = '/group/jug/ashesh/naturemethods/nicola_outofNoiseDistribution/'

def trained_on_ms(trained_ms):
    model_token = keys[trained_ms].replace('/','_')
    return f'stats_training_disentangle_{model_token}.pkl'

def prediction_for_trained_on_ms(trained_ms):
    return trained_on_ms(trained_ms).replace('stats_training_', 'pred_training_').replace('.pkl', '_1.tif')

# stats_training_disentangle_2406_D25-M3-S0-L8_17.pkl
all_ms = ['2ms', '3ms', '5ms', '20ms', '500ms']
data_dict = defaultdict(dict)
for eval_ms in all_ms:
    for trained_ms in all_ms:
        #           /group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk0/data.dset_type-3ms/stats_training_disentangle_2406_D25-M3-S0-L8_4.pkl
        with open(f'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk0/data.dset_type-{eval_ms}/{trained_on_ms(trained_ms)}', 'rb') as f:
            data = pickle.load(f)
            data_dict[eval_ms][trained_ms] = data
# eval_ms = '2ms'
# trained_ms = '500ms'
# with open(f'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8_F/data.dset_type-{eval_ms}/{trained_on_ms(trained_ms)}', 'rb') as f:
#     data = pickle.load(f)

In [ ]:
data_dict['2ms']['500ms'].keys()

In [ ]:
data_dict['2ms']['500ms']['msssim']

In [ ]:
import pandas as pd

def get_metric_data_df(metric:str,nchannels:int):
    metric_dict = [defaultdict(dict) for _ in range(nchannels)]
    metric_err_dict = [defaultdict(dict) for _ in range(nchannels)]
    for eval_ms in all_ms:
        for trained_ms in all_ms:
            tmp_arr = data_dict[eval_ms][trained_ms][metric]
            for i in range(nchannels):
                met_mean, met_std = tmp_arr[i]
                metric_dict[i][eval_ms][trained_ms] = met_mean
                metric_err_dict[i][eval_ms][trained_ms] = met_std
    
    df_arr = [pd.DataFrame.from_dict(metric_dict[ch_idx]).T for ch_idx in range(nchannels)]
    err_df_arr = [pd.DataFrame.from_dict(metric_err_dict[ch_idx]).T for ch_idx in range(nchannels)]
    return df_arr, err_df_arr



In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns

metric_list = ['rangeinvpsnr', 'ssim', 'ms3im', 'microssim', 'msssim']
metric = 'microssim'
savefig = True

assert metric in metric_list
nchannels = len(data_dict['2ms']['2ms']['rangeinvpsnr'])
df_arr, err_df_arr = get_metric_data_df(metric, nchannels)



_,ax = plt.subplots(figsize=(3*nchannels,2.4),ncols=nchannels)
for i in range(nchannels):
    ax[i].set_facecolor('lightgray')
    df_arr[i].plot(style='o-',ax=ax[i], fontsize=9)
    ax[i].set_xlabel('Evaluation Input Data', fontsize=10)
    if i == 2:
        ax[i].legend(loc='upper left', fontsize=9, title='Training Data', title_fontsize=9)

    else:
        ax[i].get_legend().remove()

    # ax[i].set_xticks(fontsize=8)
    # ax[i].set_xticks(df_arr[i].index.tolist())
    
    ax[i].grid()
    if metric != 'rangeinvpsnr':
        ax[i].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    ax[i].xaxis.set_major_locator(plt.MultipleLocator(1))
    ax[i].set_xticklabels([0] + df_arr[i].index.tolist())


if savefig:
# filename should contain all cropping information
    fname = f'{metric}_outofNoiseDistribution.png'
    fpath = os.path.join(output_data_dir, fname)
    print(fpath)
    plt.savefig(fpath, dpi=200, bbox_inches='tight')


In [ ]:
from matplotlib.ticker import FormatStrFormatter

_,ax = plt.subplots(figsize=(3*nchannels,2.4),ncols=nchannels,nrows=2)
df_ssim, _ = get_metric_data_df('ssim', nchannels)
df_msssim, _ = get_metric_data_df('msssim', nchannels)
for i in range(nchannels):
    ax[0,i].set_facecolor('lightgray')
    df_ssim[i].plot(style='o-',ax=ax[0,i], fontsize=9, markersize=4)
    # ax[0, i].set_xlabel('Evaluation Input Data', fontsize=9)

    ax[1,i].set_facecolor('lightgray')
    df_msssim[i].plot(style='o-',ax=ax[1,i], fontsize=9, markersize=4)
    ax[1, i].set_xlabel('Evaluation Input Data', fontsize=10)
    ax[1,i].xaxis.set_major_locator(plt.MultipleLocator(1))
    ax[1,i].set_xticklabels([0] + df_arr[i].index.tolist())

    ax[0,i].set_xticklabels([])
    ax[0,i].xaxis.set_major_locator(plt.MultipleLocator(1))
    for row_idx in range(2):
        ax[row_idx,i].get_legend().remove()
        ax[row_idx,i].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        ax[row_idx,i].grid()

if savefig:
# filename should contain all cropping information
    fname = f'ssim_msssim_outofNoiseDistribution.png'
    fpath = os.path.join(output_data_dir, fname)
    print(fpath)
    plt.savefig(fpath, dpi=200, bbox_inches='tight')



## Inspecting prediction

In [ ]:
from disentangle.core.tiff_reader import load_tiff
pred_data_dict = defaultdict(dict)
for eval_ms in all_ms:
    for trained_ms in all_ms:
        fpath = f'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8_F/data.dset_type-{eval_ms}/{prediction_for_trained_on_ms(trained_ms)}'
        pred_data_dict[eval_ms][trained_ms] = load_tiff(fpath)


In [ ]:
ncols = len(all_ms)
nrows = nchannels
img_sz = 3
img_idx=0
hs = 1100
ws = 100
imgN = 500
_, ax = plt.subplots(nrows, ncols, figsize=(ncols*img_sz, nrows*img_sz))
eval_ms = '500ms'
for tr_ms in all_ms:
    tmp_ = pred_data_dict[eval_ms][tr_ms]
    for ch_idx in range(nchannels):
        ax[ch_idx, all_ms.index(tr_ms)].imshow(tmp_[img_idx,hs:hs+imgN,ws:ws+imgN,ch_idx], cmap='magma')
        ax[ch_idx, all_ms.index(tr_ms)].axis('off')
        ax[0, all_ms.index(tr_ms)].set_title('Tr_' + tr_ms)

# remove space between subplots
plt.subplots_adjust(wspace=0.03, hspace=0.03)

### Evolution of prediction across noise levels.
Here, we look at some region of the prediction for different noise levels. We want to see how details appear disappear with change in noise levels

In [ ]:
pred_data_dict['2ms']['2ms'].shape

In [ ]:
import numpy as np

ncols = len(all_ms)
nrows = nchannels
img_sz = 3
img_idx=np.random.randint(pred_data_dict['2ms']['2ms'].shape[0])
imgN = 300
hs = np.random.randint(pred_data_dict['2ms']['2ms'].shape[1] -imgN)
ws = np.random.randint(pred_data_dict['2ms']['2ms'].shape[2] -imgN)
print(img_idx, hs,ws)
_, ax = plt.subplots(nrows, ncols, figsize=(ncols*img_sz, nrows*img_sz))
for ms_level in all_ms:
    tmp_ = pred_data_dict[ms_level][ms_level]
    for ch_idx in range(nchannels):
        ax[ch_idx, all_ms.index(ms_level)].imshow(tmp_[img_idx,hs:hs+imgN,ws:ws+imgN,ch_idx], cmap='magma')
        ax[ch_idx, all_ms.index(ms_level)].axis('off')
        ax[0, all_ms.index(ms_level)].set_title(ms_level)

# remove space between subplots
plt.subplots_adjust(wspace=0.03, hspace=0.03)

In [ ]:
from disentangle.config_utils import load_config


In [ ]:
from disentangle.configs.nikola_7D_config import get_config
from disentangle.data_loader.nikola_7D_rawdata_loader import get_train_val_data
from disentangle.core.data_split_type import DataSplitType
gt_dict = {}
for ms_level in all_ms:
    config = load_config(os.path.join('/group/jug/ashesh/training/disentangle/',keys[ms_level]))
    datadir = '/group/jug/ashesh/data/nikola_data/20240531/'
    data = get_train_val_data(datadir, config.data, DataSplitType.Test,
                                config.training.val_fraction, config.training.test_fraction)
    gt_dict[ms_level] = data
